In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

class TimestepEmbedding(layers.Layer):
    def __init__(self, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim

    def call(self, t):
        half = self.dim // 2
        freqs = tf.exp(tf.range(half, dtype=tf.float32) * -np.log(10000.0) / (half - 1))
        args = tf.cast(t, tf.float32)[:, None] * freqs[None, :]
        emb = tf.concat([tf.sin(args), tf.cos(args)], axis=-1)
        if self.dim % 2 == 1:
            emb = tf.pad(emb, [[0, 0], [0, 1]])
        return emb

print("TimestepEmbedding Keras Layer defined successfully.")

TimestepEmbedding Keras Layer defined successfully.


In [10]:
import os
import numpy as np
import mlflow
import mlflow.keras
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

from sklearn.decomposition import PCA

# ============================================================
# 1. CONFIG MLflow
# ============================================================

MLFLOW_URL = "https://fleecier-rufus-decadently.ngrok-free.dev"   # <--- AJUSTA ESTO
mlflow.set_tracking_uri(MLFLOW_URL)
mlflow.set_experiment("Exp3-DDPM-Conditional")


# ============================================================
# 2. CARGAR NPZ (imagenes y labels)
# ============================================================

npz_path = "./data/intermediate/pixel_art_data.npz"    # AJUSTA RUTA

data = np.load(npz_path)
images = data["images"]         # (N,16,16,3)
labels = data["labels"]         # (N,)
num_classes = len(np.unique(labels))

print("Dataset:", images.shape, "Clases:", num_classes)

images_tf = tf.convert_to_tensor(images, dtype=tf.float32)
labels_tf = tf.convert_to_tensor(labels, dtype=tf.int32)


# ============================================================
# 3. DDPM SETTINGS
# ============================================================

IMG_SHAPE = (16, 16, 3)
T = 300
BATCH_SIZE = 256
EPOCHS = 50
LR = 2e-4

# Betas lineales
beta_start = 1e-4
beta_end   = 2e-2
betas = np.linspace(beta_start, beta_end, T, dtype=np.float32)

alphas = 1.0 - betas
alphas_cumprod = np.cumprod(alphas, axis=0)

sqrt_alphas_cumprod = np.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = np.sqrt(1 - alphas_cumprod)
sqrt_recip_alphas = np.sqrt(1.0 / alphas)

betas_tf = tf.constant(betas)
alphas_tf = tf.constant(alphas)
alphas_cumprod_tf = tf.constant(alphas_cumprod)
sqrt_alphas_cumprod_tf = tf.constant(sqrt_alphas_cumprod)
sqrt_one_minus_alphas_cumprod_tf = tf.constant(sqrt_one_minus_alphas_cumprod)
sqrt_recip_alphas_tf = tf.constant(sqrt_recip_alphas)


# ============================================================
# 4. UTILIDADES DDPM
# ============================================================

def extract(a, t, x_shape):
    out = tf.gather(a, t)
    out = tf.reshape(out, (tf.shape(t)[0], 1, 1, 1))
    return tf.broadcast_to(out, x_shape)

def q_sample(x0, t, noise):
    return extract(sqrt_alphas_cumprod_tf, t, tf.shape(x0)) * x0 + \
           extract(sqrt_one_minus_alphas_cumprod_tf, t, tf.shape(x0)) * noise


# ============================================================
# 5. TIEMPO + CLASE → EMBEDDING
# ============================================================

class TimestepEmbedding(layers.Layer):
    def __init__(self, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim

    def call(self, t):
        half = self.dim // 2
        freqs = tf.exp(tf.range(half, dtype=tf.float32) * -np.log(10000.0)/(half-1))
        args = tf.cast(t, tf.float32)[:,None] * freqs[None,:]
        emb = tf.concat([tf.sin(args), tf.cos(args)], axis=-1)
        if self.dim % 2 == 1:
            emb = tf.pad(emb, [[0,0],[0,1]])
        return emb


# ============================================================
# 6. U-NET PEQUEÑO CON CONDICIONAMIENTO DE CLASE
# ============================================================

def build_unet_conditional(time_dim=64, class_dim=32, base_channels=32):
    x_in = layers.Input(shape=IMG_SHAPE, name="x")
    t_in = layers.Input(shape=(), dtype=tf.int32, name="t")
    c_in = layers.Input(shape=(), dtype=tf.int32, name="class_id")

    # --- Time Embedding ---
    time_embedding_layer = TimestepEmbedding(time_dim)
    temb = time_embedding_layer(t_in)
    temb = layers.Dense(time_dim, activation="relu")(temb)
    temb = layers.Dense(time_dim, activation="relu")(temb)

    # --- Class Embedding ---
    cemb = layers.Embedding(num_classes, class_dim)(c_in)
    cemb = layers.Dense(class_dim, activation="relu")(cemb)
    cemb = layers.Dense(time_dim, activation="relu")(cemb)

    # Fusion tiempo + clase
    cond = layers.Concatenate()([temb, cemb])
    cond = layers.Dense(time_dim, activation="relu")(cond)

    def add_cond(x, channels):
        h = layers.Dense(channels)(cond)
        h = layers.ReLU()(h) # Changed tf.nn.relu to layers.ReLU()
        h = layers.Reshape((1,1,channels))(h) # Changed tf.reshape to layers.Reshape
        return x + h

    # Encoder
    h1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(x_in)
    h1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(h1)
    h1 = add_cond(h1, base_channels)
    d1 = layers.MaxPooling2D(2)(h1)

    h2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(d1)
    h2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(h2)
    h2 = add_cond(h2, base_channels*2)
    d2 = layers.MaxPooling2D(2)(h2)

    # Bottleneck
    b = layers.Conv2D(base_channels*4, 3, padding="same", activation="relu")(d2)
    b = layers.Conv2D(base_channels*4, 3, padding="same", activation="relu")(b)
    b = add_cond(b, base_channels*4)

    # Decoder
    u2 = layers.UpSampling2D()(b)
    u2 = layers.Concatenate()([u2, h2])
    u2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(u2)
    u2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(u2)

    u1 = layers.UpSampling2D()(u2)
    u1 = layers.Concatenate()([u1, h1])
    u1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(u1)
    u1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(u1)

    out = layers.Conv2D(3, 3, padding="same", activation=None)(u1)

    return models.Model([x_in, t_in, c_in], out)


model = build_unet_conditional()
optimizer = optimizers.Adam(LR)
print(model.summary())


# ============================================================
# 7. DATASET PARA ENTRENAR (X, Y)
# ============================================================

dataset = tf.data.Dataset.from_tensor_slices((images_tf, labels_tf))
dataset = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)


# ============================================================
# 8. TRAIN STEP (CONDICIONAL)
# ============================================================

@tf.function
def train_step(x0, c):
    bsz = tf.shape(x0)[0]

    t = tf.random.uniform((bsz,), minval=0, maxval=T, dtype=tf.int32)
    noise = tf.random.normal(shape=tf.shape(x0))

    x_t = q_sample(x0, t, noise)

    with tf.GradientTape() as tape:
        noise_pred = model([x_t, t, c], training=True)
        loss = tf.reduce_mean(tf.square(noise - noise_pred))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss


# ============================================================
# 9. SAMPLING CONDICIONAL: DDPM + CLASE
# ============================================================

def sample_ddpm_class(num_samples, class_id):
    x = tf.random.normal((num_samples,) + IMG_SHAPE)
    c = tf.fill((num_samples,), class_id)

    for t in reversed(range(T)):
        t_b = tf.fill((num_samples,), t)
        eps_theta = model([x, t_b, c], training=False)

        beta_t = betas_tf[t]
        alpha_t = alphas_tf[t]
        alpha_bar_t = alphas_cumprod_tf[t]

        mean = (1/tf.sqrt(alpha_t)) * (x - beta_t * eps_theta / tf.sqrt(1-alpha_bar_t))

        if t > 0:
            x = mean + tf.sqrt(beta_t) * tf.random.normal(tf.shape(x))
        else:
            x = mean

    return tf.clip_by_value(x, 0, 1)


# ============================================================
# 10. ENTRENAMIENTO + LOGGING
# ============================================================

with mlflow.start_run(run_name="DDPM-Conditional"):

    mlflow.log_param("T", T)
    mlflow.log_param("lr", LR)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("conditioning", "class_id")

    for epoch in range(1, EPOCHS+1):
        losses = []
        for batch_x, batch_c in dataset:
            loss = train_step(batch_x, batch_c)
            losses.append(loss.numpy())

        mean_loss = float(np.mean(losses))
        print(f"Epoch {epoch} loss {mean_loss:.5f}")
        mlflow.log_metric("train_loss", mean_loss, step=epoch)

    # --------------------------------------------------------
    # 11. GENERAR SAMPLES POR CLASE
    # --------------------------------------------------------
    os.makedirs("artifacts_exp3", exist_ok=True)

    for c in range(num_classes):
        samples = sample_ddpm_class(64, c).numpy()

        fig, axes = plt.subplots(8, 8, figsize=(8,8))
        k = 0
        for i in range(8):
            for j in range(8):
                axes[i,j].imshow(samples[k])
                axes[i,j].axis("off")
                k += 1

        outpath = f"artifacts_exp3/class_{c}_grid.png"
        plt.tight_layout()
        plt.savefig(outpath)
        plt.close()
        mlflow.log_artifact(outpath)

    # --------------------------------------------------------
    # 12. CLASS-CONSISTENCY SCORE
    # (Usa tu clasificador auxiliar entrenado previamente)
    # --------------------------------------------------------
    # Suponemos que ya tienes un modelo llamado aux_model
    # Puedes cargarlo si tienes el .keras:
    # aux_model = tf.keras.models.load_model("/content/aux_classifier.keras")

    # Ejemplo dummy para no romper ejecución:
    print(">> Saltando evaluación del clasificador auxiliar (puedes agregarlo aquí).")

    # --------------------------------------------------------
    # 13. PCA REAL vs GENERADAS (por clase)
    # --------------------------------------------------------
    pca = PCA(n_components=20)
    real_flat = images.reshape(images.shape[0], -1)
    real_feats = pca.fit_transform(real_flat[:2000])

    for c in range(num_classes):
        samples = sample_ddpm_class(2000, c).numpy()
        gen_flat = samples.reshape(2000, -1)
        gen_feats = pca.transform(gen_flat)

        mu_r, mu_g = real_feats.mean(axis=0), gen_feats.mean(axis=0)
        cov_r, cov_g = np.cov(real_feats, rowvar=False), np.cov(gen_feats, rowvar=False)

        fid_like = np.linalg.norm(mu_r - mu_g) + np.linalg.norm(cov_r - cov_g, ord="fro")
        mlflow.log_metric(f"fid_like_class_{c}", float(fid_like))

        # Visual
        fig = plt.figure(figsize=(6,6))
        plt.scatter(real_feats[:,0], real_feats[:,1], s=5, alpha=0.3, label="Real")
        plt.scatter(gen_feats[:,0], gen_feats[:,1], s=5, alpha=0.3, label=f"Gen c={c}")
        plt.legend()
        plt.title(f"PCA Real vs Gen (Clase {c})")
        outp = f"artifacts_exp3/pca_class_{c}.png"
        plt.savefig(outp)
        plt.close()
        mlflow.log_artifact(outp)

    # --------------------------------------------------------
    # 14. GUARDAR MODELO
    # --------------------------------------------------------
    model.save("artifacts_exp3/ddpm_conditional.keras")
    mlflow.log_artifact("artifacts_exp3/ddpm_conditional.keras")

print("=== EXP3 (Conditional DDPM) COMPLETADO ===")

Dataset: (89400, 16, 16, 3) Clases: 5


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ t (InputLayer)      │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_id            │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ timestep_embedding… │ (None, 64)        │          0 │ t[0][0]           │
│ (TimestepEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 32)        │        160 │ class_id[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 64)        │      4,160 │ timestep_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 32)        │      1,056 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 64)        │      4,160 │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 64)        │      2,112 │ dense_22[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 128)       │          0 │ dense_21[0][0],   │
│ (Concatenate)       │                   │            │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 64)        │      8,256 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ x (InputLayer)      │ (None, 16, 16, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 32)        │      2,080 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 16, 16,    │        896 │ x[0][0]           │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 32)        │          0 │ dense_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 16, 16,    │      9,248 │ conv2d_15[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 1, 1, 32)  │          0 │ re_lu_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 16, 16,    │          0 │ conv2d_16[0][0],  │
│                     │ 32)               │            │ reshape_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 8, 8, 32)  │          0 │ add_3[0][0]       │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 64)        │      4,160 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 8, 8, 64)  │     18,496 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 64)        │          0 │ dense_26[0][0]  

 Total params: 506,851 (1.93 MB)

 Trainable params: 506,851 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1 loss 0.42445
Epoch 2 loss 0.24855
Epoch 3 loss 0.21134
Epoch 4 loss 0.18887
Epoch 5 loss 0.17522
Epoch 6 loss 0.16903
Epoch 7 loss 0.16269
Epoch 8 loss 0.15748
Epoch 9 loss 0.15276
Epoch 10 loss 0.14901
Epoch 11 loss 0.14578
Epoch 12 loss 0.14234
Epoch 13 loss 0.13794
Epoch 14 loss 0.13453
Epoch 15 loss 0.13375
Epoch 16 loss 0.12885
Epoch 17 loss 0.12675
Epoch 18 loss 0.12411
Epoch 19 loss 0.12298
Epoch 20 loss 0.12044
Epoch 21 loss 0.11907
Epoch 22 loss 0.11821
Epoch 23 loss 0.11548
Epoch 24 loss 0.11449
Epoch 25 loss 0.11306
Epoch 26 loss 0.11130
Epoch 27 loss 0.10979
Epoch 28 loss 0.11016
Epoch 29 loss 0.10923
Epoch 30 loss 0.10740
Epoch 31 loss 0.10625
Epoch 32 loss 0.10506
Epoch 33 loss 0.10394
Epoch 34 loss 0.10337
Epoch 35 loss 0.10215
Epoch 36 loss 0.10218
Epoch 37 loss 0.10045
Epoch 38 loss 0.10007
Epoch 39 loss 0.09993
Epoch 40 loss 0.09843
Epoch 41 loss 0.09779
Epoch 42 loss 0.09678
Epoch 43 loss 0.09637
Epoch 44 loss 0.09594
Epoch 45 loss 0.09505
Epoch 46 loss 